In [ ]:
import tensorflow as tf
import numpy as np
# import cv2
import cv2
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth= GoogleAuth()
gauth.LocalWebserverAuth()
drive =  GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=309331432231-6narmt12c6nugdn3mum7s18r1r6m5etp.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_3.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define the function to draw keypoints and connections
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 256, 0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_threshold) and (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 256), 2)






ValueError: ignored

In [ ]:
import os

video_folder_path = '/content/drive/MyDrive/Files'  # Orig

video_paths = []
video_labels = []

for root, dirs, files in os.walk(video_folder_path):
    for file in files:
        if file.endswith('.mp4'):  # Adjust the file extension if necessary
            video_paths.append(os.path.join(root, file))
            # Extract the label from the folder name
            video_labels.append(os.path.basename(root))
print(len(video_paths))

0
0
0


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization

with tf.device('/device:GPU:0'):
  # Load the TensorFlow Lite model (MoveNet)
  interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_3.tflite')
  interpreter.allocate_tensors()

pose_keypoints_list = []
labels_list = []

# Parameters
frames_per_second = 2  # Number of frames per second to process
minimum_video_length = 12  # Minimum length of the video in seconds
print(len(video_labels))
print(len(video_paths))
# Process each video in the dataset
count = 0
for video_path, video_label in zip(video_paths, video_labels):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Could not open video: {video_path}")
        continue
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    print(video_fps)
    if video_fps == 0:
        print('continue')
        continue
    count += 1
    print(count)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_length = frame_count / video_fps

    # Calculate the number of frames to process based on the desired frames per second and minimum video length
    total_frames_video = frames_per_second * minimum_video_length

    # Calculate the frame skip value based on the desired frames per second
    frame_skip = int(video_fps / frames_per_second)

    # Create an empty array to store the pose keypoints for the video frames
    video_pose_keypoints = np.zeros((total_frames_video, 17, 3))

    # Process each frame in the video
    frame_index = 0
    for i in range(total_frames_video):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if frame is None:
            break

        # Reshape image
        img = frame.copy()
        img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256, 256)
        input_image = tf.cast(img, dtype=tf.float32)

        # Setup input and output
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Make predictions
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

        # Store the pose keypoints for the frame
        video_pose_keypoints[i] = keypoints_with_scores[0]

        # Increment the frame index by the frame skip value
        frame_index += frame_skip

    cap.release()
    print(video_pose_keypoints.shape)
    pose_keypoints_list.append(video_pose_keypoints)
    labels_list.append(video_label)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203


In [ ]:
np.save("pose_keypoints_list_thunder.npy", pose_keypoints_list)
np.save("labels_list_thunder.npy", labels_list)
